# Predecir el partido político en función de los votos

*Como un pequeño ejemplo divertido, usaremos un conjunto de datos públicos de cómo los congresistas estadounidenses votaron sobre 17 temas diferentes en el año 1984. ¡Veamos si podemos averiguar su partido político basándonos solo en sus votos, usando una red neuronal profunda!*

*Para aquellos fuera de los Estados Unidos, nuestros dos principales partidos políticos son "demócratas" y "republicanos". En los tiempos modernos representan ideologías progresistas y conservadoras, respectivamente.*

*La política en 1984 no estaba tan polarizada como lo está hoy, pero aún así debería poder obtener más del 90% de precisión sin muchos problemas.*

*Dado que el objetivo de este ejercicio es implementar redes neuronales en Keras, te ayudaré a cargar y preparar los datos.*

*Comencemos importando el archivo CSV sin procesar usando Pandas, y hagamos un DataFrame con bonitas etiquetas de columna:*

In [1]:
import pandas as pd

feature_names =  ['party','handicapped-infants', 'water-project-cost-sharing', 
                    'adoption-of-the-budget-resolution', 'physician-fee-freeze',
                    'el-salvador-aid', 'religious-groups-in-schools',
                    'anti-satellite-test-ban', 'aid-to-nicaraguan-contras',
                    'mx-missle', 'immigration', 'synfuels-corporation-cutback',
                    'education-spending', 'superfund-right-to-sue', 'crime',
                    'duty-free-exports', 'export-administration-act-south-africa']

voting_data = pd.read_csv(r'./house-votes-84.data.txt', na_values=['?'], 
                          names = feature_names)
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


*Podemos usar describe() para tener una idea de cómo se ven los datos en conjunto:*

In [2]:
voting_data.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,435,423,387,424,424,420,424,421,420,413,428,414,404,410,418,407,331
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


*Podemos ver que faltan algunos datos con los que lidiar aquí; Algunos políticos se abstuvieron en algunas votaciones, o simplemente no estuvieron presentes cuando se realizó la votación. Simplemente eliminaremos las filas con datos faltantes para mantenerlo simple, pero en la práctica querrá asegurarse primero de que hacerlo no introduzca ningún tipo de sesgo en su análisis (si una parte se abstiene más que otra, eso podría ser problemático, por ejemplo).*

In [3]:
voting_data.dropna(inplace=True)
voting_data.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,n,y,n,y,y,y,y,n,y,n,n,y,y,n,y
freq,124,136,125,123,119,128,149,124,119,119,128,152,124,127,149,146,189


*Nuestra red neuronal necesita números normalizados, no cadenas, para funcionar. Así que reemplacemos todas las y y n con 1 y 0, y representemos a las partes como 1 y 0 también.*

In [4]:
voting_data.replace(('y', 'n'), (1, 0), inplace=True)
voting_data.replace(('democrat', 'republican'), (1, 0), inplace=True)

In [5]:
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
5,1,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
8,0,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
19,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,1,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1


*Finalmente vamos a extraer las características y etiquetas en la forma que Keras esperará:*

In [6]:
all_features = voting_data[feature_names].drop('party', axis=1).values
all_classes = voting_data['party'].values

*OK, ¡así que pruébalo! Querrá volver a la diapositiva sobre el uso de Keras con clasificación binaria: solo hay dos partes, por lo que este es un problema binario. Esto también nos ahorra la molestia de representar clases con formato "one-hot" como tuvimos que hacer con MNIST; Nuestra salida es solo un único valor 0 o 1.*

*Consulte también la diapositiva de integración de scikit_learn y utilice cross_val_score para evaluar el modelo resultante con una validación cruzada de 10 veces.*

Por cierto, si está utilizando tensorflow-gpu en una máquina con Windows, probablemente * quiera echar un vistazo a mi solución: si se encuentra con errores de asignación de memoria, hay una solución que puede usar.

Prueba tu código aquí:

In [7]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score

def create_model():
    model = Sequential()
    #16 feature inputs (votes) going into an 32-unit layer 
    model.add(Dense(32, input_dim=16, kernel_initializer='normal', activation='relu'))
    # Another hidden layer of 16 units
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    # Output layer with a binary classification (Democrat or Republican political party)
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Wrap our Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, all_features, all_classes, cv=10)
cv_scores.mean()

C:\Users\Dell Inspiron\AppData\Local\Temp\ipykernel_24956\1858574834.py:20: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)


0.9438405811786652

*¡94% sin siquiera esforzarse demasiado! ¿Lo hiciste mejor? Tal vez más neuronas, más capas o capas de abandono ayudarían aún más.*